# Example to use the custom Container for mono reconstruction

In [2]:
from lstchain.io.lstcontainers import DL1ParametersContainer

from ctapipe.utils import get_dataset_path
from ctapipe.io import HDF5TableWriter, HDF5TableReader
from ctapipe.calib import CameraCalibrator
from ctapipe.image import tailcuts_clean
from ctapipe.io import event_source
from ctapipe.image import hillas_parameters

import numpy as np
import pandas as pd

## Options

In [4]:
infile = get_dataset_path("gamma_test_large.simtel.gz")

dl1_parameters_filename = "dl1.h5"

allowed_tels = {1}  # select LST1 only
max_events = (
    300
)  # limit the number of events to analyse in files - None if no limit

cal = CameraCalibrator()

cleaning_method = tailcuts_clean
cleaning_parameters = {
    "boundary_thresh": 0,
    "picture_thresh": 6,
    "min_number_picture_neighbors": 2,
}

## R0 to DL1

In [5]:
from lstchain.reco import dl0_to_dl1

In [6]:
dl1_container = DL1ParametersContainer()

In [7]:
for i in dl1_container.keys():
    print(i)

intensity
x
y
r
phi
length
width
psi
skewness
kurtosis
disp_norm
disp_dx
disp_dy
disp_angle
disp_sign
disp_miss
src_x
src_y
time_gradient
intercept
leakage
n_islands
obs_id
event_id
gps_time
mc_energy
mc_alt
mc_az
mc_core_x
mc_core_y
mc_h_first_int
mc_type
mc_az_tel
mc_alt_tel
mc_x_max
mc_core_distance
mc_shower_primary_id
hadroness
wl
tel_id
tel_pos_x
tel_pos_y
tel_pos_z


In [10]:
with HDF5TableWriter(
    filename=dl1_parameters_filename, group_name="events", mode='w'
) as writer:

    source = event_source(infile)
    source.allowed_tels = allowed_tels
    source.max_events = max_events

    for i, event in enumerate(source):
        if i % 100 == 0:
            print(i)
        cal(event)

        for telescope_id, dl1 in event.dl1.tel.items():
            tel = event.inst.subarray.tels[telescope_id]
            camera = tel.camera
            signal_pixels = cleaning_method(
                camera, dl1.image, **cleaning_parameters
            )

            image = dl1.image
            image[~signal_pixels] = 0

            peakpos = dl1.pulse_time

            if image.sum() > 0:
                try:
                    hillas = hillas_parameters(camera, image)

                except:
                    break

                ## Fill container ##
                dl1_container.fill_mc(event)
                dl1_container.fill_hillas(hillas)
                dl1_container.fill_event_info(event)
                dl1_container.set_mc_core_distance(event, telescope_id)
                dl1_container.set_timing_features(camera, image, peakpos, hillas)
                dl1_container.set_source_camera_position(event, telescope_id)
                dl1_container.set_disp([dl1_container.src_x, dl1_container.src_y], hillas)

                ## Save parameters for later training ##
                writer.write(camera.cam_id, [dl1_container])

0


In [11]:
# The file has been created
!ls -lsh dl1.h5

304 -rw-r--r--  1 thomasvuillaume  staff   151K  6 jui 14:32 dl1.h5


## Transform a DL1 container into pandas DataFrame

In [12]:
features = ["mc_core_distance", "mc_energy"]

df = pd.DataFrame(
    [
        dl1_container.get_features(features).astype(
            np.float32
        )
    ],
    columns=features,
)
df

,mc_core_distance,mc_energy
0,129.177521,0.322498


## Transparent data reading into the container

In [14]:
from ctapipe.io import HDF5TableReader

with HDF5TableReader(dl1_parameters_filename, mode="r+") as table:
    for c in table.read("/events/LSTCam", DL1ParametersContainer()):
        print(c.disp_norm)

Table '/events/LSTCam' is missing column 'leakage' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'n_islands' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'mc_type' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'mc_shower_primary_id' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'hadroness' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'wl' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'tel_id' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'tel_pos_x' that is in container DL1ParametersContainer. It will be skipped.
Table '/events/LSTCam' is missing column 'tel_pos_y' that i

0.3845544466959033 m
0.42562420337724444 m
0.3163172942911977 m
0.5476437369940469 m
0.31884877742806345 m
0.11648724051922393 m
0.44226808748362756 m
0.5037500879480963 m
0.42478102294448955 m
0.2567060542613265 m
0.6981683571127598 m
0.21376240756997836 m
0.10270650426380194 m
0.31308882735543736 m
0.49006128385289655 m
0.3328811025559935 m
0.6845857999628625 m
0.3641529388515587 m
0.3629023944371933 m
0.7308373767973714 m
0.6911421061088385 m
0.5835773596016229 m
0.7339108018657268 m
0.5069680867622501 m
0.25366379034430186 m
0.28173881691977415 m
0.07919617097462123 m
0.24058246882727433 m
0.6374814336780688 m
0.4430826031667524 m
0.6125365664856339 m
1.3071950045553462 m
1.0615231977262378 m
0.9800461151743801 m
0.5008866879849349 m
0.7237337498559453 m
0.39119847402965846 m
0.15862952639852898 m
0.45996782823114096 m
0.13833792924832045 m
0.45543500381602126 m
0.34146530310511003 m
0.2803431832015625 m
0.15165536181376893 m
0.32404518799291665 m
0.9942239796854567 m
0.81814779173

## The hdf5 file is also very easy to read with pandas

In [15]:
import pandas as pd

pd.read_hdf(dl1_parameters_filename, key="events/LSTCam")

,disp_angle,disp_dx,disp_dy,disp_miss,disp_norm,disp_sign,event_id,gps_time,intensity,intercept,...,phi,psi,r,skewness,src_x,src_y,time_gradient,width,x,y
0,1.448087,-0.047070,-0.381663,0.077688,0.384554,-1.0,153614,57576.867777,85.116041,8.762430,...,2.063250,1.244666,0.964152,-0.179795,-0.502911,0.467924,-6.817286,0.024291,-0.455841,0.849587
1,0.801960,-0.295936,-0.305905,0.057078,0.425624,-1.0,192801,57576.877092,154.176998,9.495189,...,2.903815,0.936471,1.055811,-0.212384,-1.322041,-0.057215,9.959528,0.037709,-1.026105,0.248690
2,0.071120,-0.315518,-0.022477,0.137342,0.316317,-1.0,192801,57576.877092,306.658826,9.615328,...,-3.107094,-0.378020,1.007123,0.429407,-1.322041,-0.057215,5.968673,0.054983,-1.006523,-0.034738
3,0.322652,-0.519384,-0.173648,0.201767,0.547644,-1.0,192801,57576.877092,90.034911,8.252871,...,2.997538,-0.054665,0.811058,0.379953,-1.322041,-0.057215,8.430052,0.033023,-0.802657,0.116433
4,0.823835,-0.216630,-0.233958,0.016406,0.318849,-1.0,192801,57576.877092,113.531421,8.637336,...,2.983046,0.875311,1.119452,0.787276,-1.322041,-0.057215,1.446689,0.022988,-1.105411,0.176742
5,-0.235149,0.113281,-0.027140,0.057545,0.116487,1.0,222202,57576.877237,191.975511,9.245389,...,-0.101789,0.281541,0.745029,-0.300447,0.854454,-0.102845,6.854362,0.035884,0.741173,-0.075705
6,1.370050,0.088188,0.433386,0.012320,0.442268,1.0,222202,57576.877237,222.821548,9.896140,...,-0.610591,1.342191,0.935258,0.074430,0.854454,-0.102845,-0.448335,0.034982,0.766266,-0.536232
7,0.445018,0.454686,0.216852,0.032830,0.503750,1.0,222202,57576.877237,192.102860,9.225058,...,-0.674561,0.510236,0.511880,-0.147983,0.854454,-0.102845,-6.900914,0.030610,0.399768,-0.319697
8,-1.236051,0.139553,-0.401203,0.052629,0.424781,1.0,380915,57576.878458,530.255864,10.853515,...,-1.596808,-1.360268,0.393724,-0.035591,0.129313,-0.794794,-4.025235,0.035988,-0.010240,-0.393591
9,1.200404,-0.092923,-0.239298,0.068146,0.256706,-1.0,380915,57576.878458,146.325133,9.717524,...,-1.190233,1.469090,0.598302,0.082425,0.129313,-0.794794,7.964812,0.026976,0.222235,-0.555496


In [17]:
rm dl1.h5